# Initializing the SysML v2 API

In [1]:
from __future__ import print_function

import time
import requests
from pprint import pprint
import pandas as pd
import json
from datetime import datetime

#host = "<specify protocol://host:port of the server that is a provider of the SysML v2 REST/HTTP API"
host = "http://sysmlapiserver:9000"

# Get projects

In [4]:
projects_url = f"{host}/projects" 
projects_response = requests.get(projects_url)

if projects_response.status_code == 200:
    projects = projects_response.json()
    projects_data = list(map(lambda b: {'Project Name':b['name'], 'Project ID':b['@id']}, projects))
    df = pd.DataFrame.from_records(projects_data).style.hide(axis='index')
    # if len(projects_data) > 0:
    #     df = df.style.sort_values(by='Project Name')
    display(df)
else:
    pprint("Problem in fetching projects")

Project Name,Project ID
Spacecraft project with branches and tags - 2025-04-10 21:32:01.587889,d893a70a-4c18-469e-b4a5-7597424f1829


# Create a new project

In [3]:
timestamp = datetime.now()
project_name = f"Spacecraft project with branches and tags - {timestamp}"
project_data = {
  "@type":"Project",
  "name": project_name,
  "description": "Spacecraft project with multiple commits, branches, and tags"
}

project_post_url = f"{host}/projects" 

project_post_response = requests.post(project_post_url, 
                                      headers={"Content-Type": "application/json"}, 
                                      data=json.dumps(project_data))

project_id = ""

if project_post_response.status_code == 200:
    project_response_json = project_post_response.json()
    pprint(project_response_json)
    project_id = project_response_json['@id']
    project_name = project_response_json['name']
else:
    pprint(f"Problem in creating a new Spacecraft project at {timestamp}")
    pprint(project_post_response)

{'@id': 'd893a70a-4c18-469e-b4a5-7597424f1829',
 '@type': 'Project',
 'created': '2025-04-10T21:32:01.636403Z',
 'defaultBranch': {'@id': 'd5f63dc9-fbbd-4ea3-a236-6dc3e1731975'},
 'description': 'Spacecraft project with multiple commits, branches, and tags',
 'name': 'Spacecraft project with branches and tags - 2025-04-10 '
         '21:32:01.587889'}


# Get branches

In [5]:
branches_url = f"{host}/projects/{project_id}/branches" 
branches_response = requests.get(branches_url)

if branches_response.status_code == 200:
    branches = branches_response.json()
    branches_data = list(map(lambda b: {'Branch Name':b['name'], 'Branch ID':b['@id'], 'Ref Commit (head)':b['head']}, branches))

    df = pd.DataFrame.from_records(branches_data).sort_values(by='Branch Name').style.hide(axis='index')
    display(df)
else:
    pprint(f"Problem in fetching branches of Spacecraft project {project_id}")
    pprint(branches_response)

Branch Name,Branch ID,Ref Commit (head)
main,d5f63dc9-fbbd-4ea3-a236-6dc3e1731975,None


# Get tags

In [6]:
tags_url = f"{host}/projects/{project_id}/tags" 
tags_response = requests.get(tags_url)

if tags_response.status_code == 200:
    tags = tags_response.json()
    pprint(tags)
    tags_data = list(map(lambda b: {'Tag Name':b['name'], 'Tag ID':b['@id'], 'Tagged Commit':b['taggedCommit']}, tags))

    df = pd.DataFrame.from_records(tags_data).style.hide(axis='index')
    display(df)
else:
    pprint(f"Problem in fetching tags of Spacecraft project {project_id}")
    pprint(tags_response)

[]


# Create 1st Commit

In [7]:
commit_body = {
  "@type": "Commit",
  "change": [
    {
      "@type": "DataVersion",
      "payload": {
        "@type": "PartDefinition",
        "name":"Spacecraft System"
      }
    },
    {
      "@type": "DataVersion",
      "payload": {
        "@type": "PartDefinition",
        "name":"Payload System"
      }
    },
    {
      "@type": "DataVersion",
      "payload": {
        "@type": "PartDefinition",
        "name":"Propulsion System"
      }
    }
  ]
}

commit_post_url = f"{host}/projects/{project_id}/commits" 

commit_post_response = requests.post(commit_post_url, 
                                      headers={"Content-Type": "application/json"}, 
                                      data=json.dumps(commit_body))

commit1_id = ""

if commit_post_response.status_code == 200:
    commit_response_json = commit_post_response.json()
    pprint(commit_response_json)
    commit1_id = commit_response_json['@id']
else:
    pprint(f"Problem in creating a new commit in Spacecraft project {project_id}")
    pprint(commit_post_response)

{'@id': '06c61b49-a37a-4192-a674-b1b531fa33b1',
 '@type': 'Commit',
 'created': '2025-04-10T21:32:31.970757Z',
 'description': None,
 'owningProject': {'@id': 'd893a70a-4c18-469e-b4a5-7597424f1829'},
 'previousCommit': None}


In [8]:
element_get_url = f"{host}/projects/{project_id}/commits/{commit1_id}/elements" 

element_get_response = requests.get(element_get_url)

if element_get_response.status_code == 200:
    elements = element_get_response.json()
    elements_data = list(map(lambda b: {'Element Name':b['name'], 'Element ID':b['@id']}, elements))
    df = pd.DataFrame.from_records(elements_data).sort_values(by='Element Name').style.hide(axis='index')
    display(df)
else:
    pprint(f"Problem in fetching elements in the Spacecraft project {project_id} at commit {commit1_id}.")
    pprint(element_get_response)

Element Name,Element ID
Payload System,aa7b80a6-6a2c-4048-a835-ae404ab29e7d
Propulsion System,dd59488a-e401-4f51-b06a-ed995ee820aa
Spacecraft System,fa313a79-c315-43d4-b6bf-9b398ec97f33


# Create 2nd Commit

In [9]:
commit_body = {
  "@type": "Commit",
  "change": [
    {
      "@type": "DataVersion",
      "payload": {
        "@type": "PartDefinition",
        "name":"Avionics System"
      }
    },
    {
      "@type": "DataVersion",
      "payload": {
        "@type": "PartDefinition",
        "name":"Power System"
      }
    }
  ],
  "previousCommit": {
      "@id": commit1_id
    }
}

commit_post_url = f"{host}/projects/{project_id}/commits" 

commit_post_response = requests.post(commit_post_url, 
                                      headers={"Content-Type": "application/json"}, 
                                      data=json.dumps(commit_body))

commit2_id = ""

if commit_post_response.status_code == 200:
    commit_response_json = commit_post_response.json()
    pprint(commit_response_json)
    commit2_id = commit_response_json['@id']
else:
    pprint(f"Problem in creating a new commit in Spacecraft project {project_id}")
    pprint(commit_post_response)

{'@id': '65fee900-6a08-4840-8e46-27fadae09338',
 '@type': 'Commit',
 'created': '2025-04-10T21:32:48.89137Z',
 'description': None,
 'owningProject': {'@id': 'd893a70a-4c18-469e-b4a5-7597424f1829'},
 'previousCommit': {'@id': '06c61b49-a37a-4192-a674-b1b531fa33b1'}}


# Get all elements after the 2nd commit

In [10]:
element_get_url = f"{host}/projects/{project_id}/commits/{commit2_id}/elements" 

element_get_response = requests.get(element_get_url)

if element_get_response.status_code == 200:
    elements = element_get_response.json()
    elements_data = list(map(lambda b: {'Element Name':b['name'], 'Element ID':b['@id']}, elements))
    df = pd.DataFrame.from_records(elements_data).sort_values(by='Element Name').style.hide(axis='index')
    display(df)
else:
    pprint(f"Problem in fetching elements in the Spacecraft project {project_id} at commit {commit2_id}.")
    pprint(element_get_response)

Element Name,Element ID
Avionics System,5b665fc8-d849-40fe-aa19-f266d3fdd075
Payload System,aa7b80a6-6a2c-4048-a835-ae404ab29e7d
Power System,2f22316b-e7db-4f09-955e-e16415c6bfae
Propulsion System,dd59488a-e401-4f51-b06a-ed995ee820aa
Spacecraft System,fa313a79-c315-43d4-b6bf-9b398ec97f33


# Create a tag

In [11]:
tag_body = {
    "@type": "Tag",
    "name": "Spacecraft Internal Release 0.1",
    "taggedCommit": {
      "@id": commit2_id
    }
}

tag_post_url = f"{host}/projects/{project_id}/tags" 

tag_post_response = requests.post(tag_post_url, 
                                  headers={"Content-Type": "application/json"}, 
                                  data=json.dumps(tag_body))

tag1_id = ""

if tag_post_response.status_code == 200:
    tag_response_json = tag_post_response.json()
    pprint(tag_response_json)
    tag1_id = tag_response_json['@id']
else:
    pprint(f"Problem in creating a new tag in Spacecraft project {project_id}.")
    pprint(tag_post_response)

{'@id': '7873206c-c481-46c9-af8b-815d3e47a25f',
 '@type': 'Tag',
 'created': '2025-04-10T21:32:55.490596Z',
 'name': 'Spacecraft Internal Release 0.1',
 'owningProject': {'@id': 'd893a70a-4c18-469e-b4a5-7597424f1829'},
 'referencedCommit': {'@id': '65fee900-6a08-4840-8e46-27fadae09338'},
 'taggedCommit': {'@id': '65fee900-6a08-4840-8e46-27fadae09338'}}


# Get Tags

In [12]:
tags_url = f"{host}/projects/{project_id}/tags" 
tags_response = requests.get(tags_url)

if tags_response.status_code == 200:
    tags = tags_response.json()
    tags_data = list(map(lambda b: {'Tag Name':b['name'], 'Tag ID':b['@id'], 'Tagged Commit':b['taggedCommit']}, tags))

    df = pd.DataFrame.from_records(tags_data).style.hide(axis='index')
    display(df)
else:
    pprint(f"Problem in fetching tags in Spacecraft project {project_id}.")
    pprint(tag_post_response)

Tag Name,Tag ID,Tagged Commit
Spacecraft Internal Release 0.1,7873206c-c481-46c9-af8b-815d3e47a25f,{'@id': '65fee900-6a08-4840-8e46-27fadae09338'}


# Create a new branch at the second commit (also tagged as 0.1)

In [13]:
branch_body = {
  "@type": "Branch",
  "name": "develop",
  "head": {
      "@id": commit2_id
    }
}

branch_post_url = f"{host}/projects/{project_id}/branches" 

branch_post_response = requests.post(branch_post_url, 
                                      headers={"Content-Type": "application/json"}, 
                                      data=json.dumps(branch_body))

branch_develop_id = ""

if branch_post_response.status_code == 200:
    branch_response_json = branch_post_response.json()
    pprint(branch_response_json)
    branch_develop_id = branch_response_json['@id']
else:
    pprint(f"Problem in creating a new branch in Spacecraft project {project_id}")
    pprint(branch_post_response)

{'@id': '793dc363-566e-4356-acb8-b0610afa3cfb',
 '@type': 'Branch',
 'created': '2025-04-10T21:33:02.621274Z',
 'head': {'@id': '65fee900-6a08-4840-8e46-27fadae09338'},
 'name': 'develop',
 'owningProject': {'@id': 'd893a70a-4c18-469e-b4a5-7597424f1829'},
 'referencedCommit': {'@id': '65fee900-6a08-4840-8e46-27fadae09338'}}


# Get branches

In [14]:
branches_url = f"{host}/projects/{project_id}/branches" 
branches_response = requests.get(branches_url)

if branches_response.status_code == 200:
    branches = branches_response.json()
    branches_data = list(map(lambda b: {'Branch Name':b['name'], 'Branch ID':b['@id'], 'Ref Commit (head)':b['head']}, branches))

    df = pd.DataFrame.from_records(branches_data).sort_values(by='Branch Name').style.hide(axis='index')
    display(df)
else:
    pprint(f"Problem in fetching branches in Spacecraft project {project_id}")
    pprint(branches_response)

Branch Name,Branch ID,Ref Commit (head)
develop,793dc363-566e-4356-acb8-b0610afa3cfb,{'@id': '65fee900-6a08-4840-8e46-27fadae09338'}
main,d5f63dc9-fbbd-4ea3-a236-6dc3e1731975,{'@id': '65fee900-6a08-4840-8e46-27fadae09338'}


# Create a new commit in the develop branch

In [15]:
commit_body = {
  "@type": "Commit",
  "change": [
    {
      "@type": "DataVersion",
      "payload": {
        "@type": "PartDefinition",
        "name":"GN & C System"
      }
    }
  ],
  "previousCommit": {
      "@id": commit2_id
    }
}

commit_post_url = f"{host}/projects/{project_id}/commits" 

commit_post_response = requests.post(commit_post_url, 
                                      headers={"Content-Type": "application/json"}, 
                                      data=json.dumps(commit_body),
                                      params={"branchId":branch_develop_id})

commit3_id = ""

if commit_post_response.status_code == 200:
    commit_response_json = commit_post_response.json()
    pprint(commit_response_json)
    commit3_id = commit_response_json['@id']
else:
    pprint(f"Problem in creating a new commit in Spacecraft project {project_id}")
    pprint(commit_post_response)

{'@id': '43d8e05e-d9f6-48a3-abbf-f22a722b67b4',
 '@type': 'Commit',
 'created': '2025-04-10T21:33:11.023767Z',
 'description': None,
 'owningProject': {'@id': 'd893a70a-4c18-469e-b4a5-7597424f1829'},
 'previousCommit': {'@id': '65fee900-6a08-4840-8e46-27fadae09338'}}


# Get Branches

In [16]:
branches_url = f"{host}/projects/{project_id}/branches" 
branches_response = requests.get(branches_url)

if branches_response.status_code == 200:
    branches = branches_response.json()
    branches_data = list(map(lambda b: {'Branch Name':b['name'], 'Branch ID':b['@id'], 'Ref Commit (head)':b['head']}, branches))

    df = pd.DataFrame.from_records(branches_data).sort_values(by='Branch Name').style.hide(axis='index')
    display(df)
else:
    pprint(f"Problem in fetching branches in Spacecraft project {project_id}")
    pprint(commit_post_response)

Branch Name,Branch ID,Ref Commit (head)
develop,793dc363-566e-4356-acb8-b0610afa3cfb,{'@id': '43d8e05e-d9f6-48a3-abbf-f22a722b67b4'}
main,d5f63dc9-fbbd-4ea3-a236-6dc3e1731975,{'@id': '65fee900-6a08-4840-8e46-27fadae09338'}


# Get all elements as of the latest commit on main branch

In [17]:
pprint(f"Fetching elements at commit {commit2_id}")
element_get_url = f"{host}/projects/{project_id}/commits/{commit2_id}/elements" 

element_get_response = requests.get(element_get_url)

if element_get_response.status_code == 200:
    elements = element_get_response.json()
    elements_data = list(map(lambda b: {'Element Name':b['name'], 'Element ID':b['@id']}, elements))
    df = pd.DataFrame.from_records(elements_data).sort_values(by='Element Name').style.hide(axis='index')
    display(df)
else:
    pprint(f"Problem in fetching elements in the Spacecraft project {project_id} at commit {commit2_id}")
    pprint(element_get_response)

'Fetching elements at commit 65fee900-6a08-4840-8e46-27fadae09338'


Element Name,Element ID
Avionics System,5b665fc8-d849-40fe-aa19-f266d3fdd075
Payload System,aa7b80a6-6a2c-4048-a835-ae404ab29e7d
Power System,2f22316b-e7db-4f09-955e-e16415c6bfae
Propulsion System,dd59488a-e401-4f51-b06a-ed995ee820aa
Spacecraft System,fa313a79-c315-43d4-b6bf-9b398ec97f33


# Get all elements as of the latest commit on develop branch

In [18]:
pprint(f"Fetching elements at commit {commit3_id}")
element_get_url = f"{host}/projects/{project_id}/commits/{commit3_id}/elements" 

element_get_response = requests.get(element_get_url)

if element_get_response.status_code == 200:
    elements = element_get_response.json()
    elements_data = list(map(lambda b: {'Element Name':b['name'], 'Element ID':b['@id']}, elements))
    df = pd.DataFrame.from_records(elements_data).sort_values(by='Element Name').style.hide(axis='index')
    display(df)
else:
    pprint(f"Problem in fetching elements in the Spacecraft project {project_id} at commit {commit3_id}")
    pprint(element_get_response)

'Fetching elements at commit 43d8e05e-d9f6-48a3-abbf-f22a722b67b4'


Element Name,Element ID
Avionics System,5b665fc8-d849-40fe-aa19-f266d3fdd075
GN & C System,37923548-0fae-44b1-b1a2-bab6426d1ec4
Payload System,aa7b80a6-6a2c-4048-a835-ae404ab29e7d
Power System,2f22316b-e7db-4f09-955e-e16415c6bfae
Propulsion System,dd59488a-e401-4f51-b06a-ed995ee820aa
Spacecraft System,fa313a79-c315-43d4-b6bf-9b398ec97f33


# Create a tag at the last commit on develop branch

In [19]:
tag_body = {
    "@type": "Tag",
    "name": "Spacecraft Internal Release 0.2 build 1",
    "taggedCommit": {
      "@id": commit3_id
    }
}

tag_post_url = f"{host}/projects/{project_id}/tags" 

tag_post_response = requests.post(tag_post_url, 
                                  headers={"Content-Type": "application/json"}, 
                                  data=json.dumps(tag_body))

tag1_id = ""

if tag_post_response.status_code == 200:
    tag_response_json = tag_post_response.json()
    pprint(tag_response_json)
    tag1_id = tag_response_json['@id']
else:
    pprint(f"Problem in creating a new tag in Spacecraft project {project_id}.")
    pprint(tag_post_response)

{'@id': '99f9951e-e1f9-4ccc-a31c-52a73e2c2ec8',
 '@type': 'Tag',
 'created': '2025-04-10T21:33:28.874488Z',
 'name': 'Spacecraft Internal Release 0.2 build 1',
 'owningProject': {'@id': 'd893a70a-4c18-469e-b4a5-7597424f1829'},
 'referencedCommit': {'@id': '43d8e05e-d9f6-48a3-abbf-f22a722b67b4'},
 'taggedCommit': {'@id': '43d8e05e-d9f6-48a3-abbf-f22a722b67b4'}}


# Get all tags

In [20]:
tags_url = f"{host}/projects/{project_id}/tags" 
tags_response = requests.get(tags_url)

if tags_response.status_code == 200:
    tags = tags_response.json()
    tags_data = list(map(lambda b: {'Tag Name':b['name'], 'Tag ID':b['@id'], 'Tagged Commit':b['taggedCommit']}, tags))

    df = pd.DataFrame.from_records(tags_data).style.hide(axis='index')
    display(df)
else:
    pprint(f"Problem in fetching tags in Spacecraft project {project_id}.")
    pprint(tag_post_response)

Tag Name,Tag ID,Tagged Commit
Spacecraft Internal Release 0.1,7873206c-c481-46c9-af8b-815d3e47a25f,{'@id': '65fee900-6a08-4840-8e46-27fadae09338'}
Spacecraft Internal Release 0.2 build 1,99f9951e-e1f9-4ccc-a31c-52a73e2c2ec8,{'@id': '43d8e05e-d9f6-48a3-abbf-f22a722b67b4'}
